### test_dataset preprocessing  
1. 년도별 데이터셋 dict 형식으로 변경 dict[기업별][년도별][문장리스트]  
2. 뉴스 19년도부터의 데이터로 인해 잡플래닛 19년도 이하의 데이터들 제외  

In [1]:
import pandas as pd

In [7]:
df_2019 = pd.read_csv('./kosdaq_data/2019_kosdaq.csv', on_bad_lines='skip')
df_2020 = pd.read_csv('./kosdaq_data/2020_kosdaq.csv', on_bad_lines='skip')
df_2021 = pd.read_csv('./kosdaq_data/2021_kosdaq.csv', on_bad_lines='skip')
df_2022 = pd.read_csv('./kosdaq_data/2022_kosdaq.csv', on_bad_lines='skip')
df_2023 = pd.read_csv('./kosdaq_data/2023_kosdaq.csv', on_bad_lines='skip')
df_2024 = pd.read_csv('./kosdaq_data/2024_kosdaq.csv', on_bad_lines='skip')

df_news = pd.concat([df_2019,df_2020,df_2021,df_2022,df_2023,df_2024])

In [9]:
len(df_news)

594200

In [3]:
df_news = df_news.drop_duplicates(subset='content')
data = []

for index, row in df_news.iterrows():
    sentences = row['content'].split('&&')
    for sentence in sentences:
        if len(sentence) > 40:
            data.append((row['company'], row['date'],row['url'], sentence))
df_news = pd.DataFrame(data, columns=['Company','Date','URL', 'Sentence'])

In [4]:
len(df_news)

5068917

In [5]:
df_news = df_news.drop_duplicates(subset='Sentence')

In [6]:
len(df_news)

4571733

In [ ]:
job_df = pd.read_csv('./data/kosdaq_date_pros_cons_jobplanet_result.csv')

In [ ]:
# 기업별 날짜별 문장을 담을 딕셔너리
company_date_sentence = {}

# 뉴스
for _,token in df_news.iterrows():
    company_name = token['Company']
    sentences = token['Sentence']
    year = token['Date'].split('.')[0]

    if company_name not in company_date_sentence:
        company_date_sentence[company_name] = {year: [sentences]}
    else:
        if year not in company_date_sentence[company_name]:
            company_date_sentence[company_name][year] = [sentences]
        else:
            company_date_sentence[company_name][year].extend([sentences])

# 잡플래닛
for _, token in job_df.iterrows():
    company_name = token['기업명']
    pos_sentence = token['장점 리뷰']
    neg_sentence = token['단점 리뷰']
    year = token['날짜']
    if company_name not in company_date_sentence:
        company_date_sentence[company_name] = {year: [pos_sentence, neg_sentence]}
    else:
        if year not in company_date_sentence[company_name]:
            company_date_sentence[company_name][year] = [pos_sentence, neg_sentence]
        else:
            company_date_sentence[company_name][year].extend([pos_sentence, neg_sentence])

In [ ]:
# 잡플래닛 18년도 이하 데이터 제외
for company in company_date_sentence:
    for year in range(2014,2019):
        if str(year) in company_date_sentence[company]:
            del company_date_sentence[company][str(year)]

In [ ]:
import json

with open("total_kosdaq.json", "w") as f:
    json.dump(company_date_sentence, f)